# CONFLUENCE Tutorial - 10: CAMELS Large Sample Study (Multi-Basin Streamflow Analysis)

## Introduction

This tutorial demonstrates systematic streamflow modeling across multiple watersheds using the CAMELS spat dataset (Knoben et al., 2025). Unlike previous tutorials focused on point-scale processes, this analysis addresses watershed-scale streamflow prediction across diverse environmental conditions.

### CAMELS Spat Dataset

The CAMELS Spat dataset provides standardized data for 671 watersheds across the contiguous United States, with global extensions covering additional regions. The dataset includes gridded meteorological forcing, quality-controlled daily discharge observations, and comprehensive catchment attributes. Watersheds range from 4 to 25,000 km² and span diverse climate and physiographic conditions while maintaining focus on near-natural basins with minimal human impact.

### Streamflow Modeling Challenges

Streamflow represents the integrated watershed response to precipitation, evapotranspiration, groundwater interactions, and routing processes. Multi-basin analysis presents challenges including spatial heterogeneity across landscapes, temporal dynamics spanning multiple timescales, and scale interactions between different hydrological processes.

### Research Objectives

This tutorial addresses fundamental questions about dominant controls on streamflow generation, model performance variations across environmental gradients, parameter transferability between watersheds, and streamflow sensitivity to climate variability. The analysis employs multiple performance metrics including Nash-Sutcliffe efficiency, Kling-Gupta efficiency, and bias assessment.

### Methodological Framework

The approach involves strategic site selection across environmental gradients, standardized model configuration for diverse basin characteristics, batch processing execution, and systematic performance evaluation. Sites are selected to represent climate diversity, physiographic variation, and multiple watershed scales while ensuring adequate data quality.

### CONFLUENCE Advantages

CONFLUENCE provides consistent methodology across watersheds, automated processing capabilities, and systematic quality control. The framework emphasizes process-based modeling with flexible structure adaptable to different watershed characteristics and comprehensive uncertainty assessment.

### Expected Outcomes

This tutorial demonstrates watershed-scale configuration, streamflow validation through observed-simulated comparisons, performance analysis across sites, regional pattern identification, and process diagnostics. Results contribute to improved understanding of hydrological controls, enhanced model development, and applications in water resources management and climate impact assessment.

## Step 1: Multi-Basin Streamflow Experimental Design and Site Selection
Transitioning from the FLUXNET energy balance and NORSWE snow focus to systematic streamflow hydrology simulations, this step establishes the foundation for large sample hydrological modeling using the comprehensive CAMELS-Spat dataset. We demonstrate how CONFLUENCE's workflow efficiency enables systematic streamflow evaluation across the full spectrum of North American hydroclimate.

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import yaml
from datetime import datetime
import seaborn as sns
import warnings

# Set up plotting style for watershed visualization
plt.style.use('default')
sns.set_palette("viridis")
%matplotlib inline
confluence_path = Path('../').resolve()

# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/anvil/scratch/x-deythorsson/CONFLUENCE_data')  # Update this path
#CONFLUENCE_DATA_DIR = Path('/path/to/your/CONFLUENCE_data') 

# =============================================================================
# CAMELS-SPAT TEMPLATE CONFIGURATION
# =============================================================================

# Load streamflow configuration template or create from base template
streamflow_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_template.yaml'
with open(streamflow_config_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update for CAMELS-SPAT tutorial-specific settings
config_updates = {
    'CONFLUENCE_CODE_DIR': str(CONFLUENCE_CODE_DIR),
    'CONFLUENCE_DATA_DIR': str(CONFLUENCE_DATA_DIR),
    'DOMAIN_NAME': 'camelsspat_template',
    'EXPERIMENT_ID': 'run_1',
    'EXPERIMENT_TIME_START': '2005-01-01 01:00',
    'EXPERIMENT_TIME_END': '2015-12-31 23:00',  # 10-year period for streamflow analysis
}

config_dict.update(config_updates)

# Save CAMELS-SPAT configuration template
camelsspat_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_camelsspat_template.yaml'
with open(camelsspat_config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

print(f"CAMELS-SPAT template configuration saved: {camelsspat_config_path}")

# =============================================================================
# LOAD AND EXAMINE CAMELS-SPAT WATERSHED DATASET
# =============================================================================

print(f"\nLoading CAMELS-SPAT Watershed Database...")

# Load the CAMELS-SPAT watersheds database
try:
    camelsspat_df = pd.read_csv('camels-spat-metadata.csv')
    print(f"Successfully loaded CAMELS-SPAT database: {len(camelsspat_df)} watersheds available")
except FileNotFoundError:
    print(f"CAMELS-SPAT database not found, creating demonstration dataset...")
    
    # Create demonstration CAMELS-SPAT dataset for tutorial
    np.random.seed(42)
    n_watersheds = 100
    
    # Generate realistic North American watershed locations
    # Focus on major watershed regions with good streamflow data
    regions = [
        {'name': 'Pacific_Northwest', 'lat_range': (42, 49), 'lon_range': (-125, -117), 'n': 20},
        {'name': 'Rocky_Mountains', 'lat_range': (37, 48), 'lon_range': (-115, -105), 'n': 25},
        {'name': 'Great_Plains', 'lat_range': (35, 45), 'lon_range': (-105, -95), 'n': 15},
        {'name': 'Southeastern_US', 'lat_range': (30, 40), 'lon_range': (-95, -80), 'n': 20},
        {'name': 'Northeastern_US', 'lat_range': (40, 47), 'lon_range': (-80, -67), 'n': 15},
        {'name': 'California', 'lat_range': (32, 42), 'lon_range': (-125, -114), 'n': 5}
    ]
    
    watersheds_data = []
    watershed_id = 1
    
    for region in regions:
        for i in range(region['n']):
            lat = np.random.uniform(region['lat_range'][0], region['lat_range'][1])
            lon = np.random.uniform(region['lon_range'][0], region['lon_range'][1])
            
            # Area based on typical CAMELS watersheds (log-normal distribution)
            area = np.random.lognormal(np.log(200), 1.2)
            area = np.clip(area, 10, 15000)  # Clip to CAMELS range
            
            # Elevation varies by region and affects streamflow characteristics
            if region['name'] == 'Rocky_Mountains':
                elevation = np.random.uniform(1200, 3500)
            elif region['name'] == 'Pacific_Northwest':
                elevation = np.random.uniform(300, 2200)
            elif region['name'] == 'California':
                elevation = np.random.uniform(200, 2800)
            else:
                elevation = np.random.uniform(50, 1200)
            
            # Climate characteristics affecting streamflow
            if region['name'] in ['Pacific_Northwest', 'Southeastern_US']:
                map_precip = np.random.uniform(800, 2500)  # Wet regions
                mat_temp = np.random.uniform(5, 18)
            elif region['name'] == 'Great_Plains':
                map_precip = np.random.uniform(300, 800)   # Dry regions
                mat_temp = np.random.uniform(8, 16)
            else:
                map_precip = np.random.uniform(400, 1500)  # Mixed
                mat_temp = np.random.uniform(-2, 15)
            
            # Derived characteristics
            aridity = map_precip / (max(0.1, mat_temp + 5) * 365)  # Aridity index
            seasonality = np.random.uniform(0.2, 0.8)  # Precipitation seasonality
            
            # Forest fraction varies by region
            if region['name'] in ['Pacific_Northwest', 'Southeastern_US', 'Northeastern_US']:
                forest_frac = np.random.uniform(0.4, 0.9)
            else:
                forest_frac = np.random.uniform(0.05, 0.4)
            
            # Scale classification based on area
            if area < 100:
                scale = 'headwater'
            elif area < 1000:
                scale = 'meso'
            else:
                scale = 'macro'
            
            # Streamflow characteristics
            mean_q = area * map_precip * 0.001 * np.random.uniform(0.2, 0.8)  # Rough runoff coefficient
            baseflow_index = np.random.uniform(0.2, 0.8)
            
            # Create watershed entry
            watershed = {
                'station_id': f"CAMELS_{watershed_id:04d}",
                'station_name': f"{region['name']}_Basin_{i+1:03d}",
                'lat': round(lat, 4),
                'lon': round(lon, 4),
                'drainage_area': round(area, 1),
                'elevation_mean': round(elevation, 0),
                'p_mean': round(map_precip, 0),  # Mean annual precipitation
                't_mean': round(mat_temp, 1),    # Mean annual temperature
                'aridity': round(aridity, 3),
                'seasonality': round(seasonality, 3),
                'forest_frac': round(forest_frac, 3),
                'q_mean': round(mean_q, 2),
                'baseflow_index': round(baseflow_index, 3),
                'scale': scale,
                'region': region['name'],
                'data_length': np.random.randint(10, 30),  # Years of data
                'data_quality': np.random.choice(['excellent', 'good', 'fair'], p=[0.3, 0.5, 0.2])
            }
            
            # Add CONFLUENCE formatting
            buffer = 0.05
            watershed['BOUNDING_BOX_COORDS'] = f"{lat + buffer}/{lon - buffer}/{lat - buffer}/{lon + buffer}"
            watershed['POUR_POINT_COORDS'] = f"{lat}/{lon}"
            watershed['Watershed_Name'] = watershed['station_id'].replace(' ', '_')
            
            watersheds_data.append(watershed)
            watershed_id += 1
    
    camelsspat_df = pd.DataFrame(watersheds_data)
    
    # Save demonstration dataset
    camelsspat_df.to_csv('camels-spat-metadata.csv', index=False)
    print(f"Created demonstration CAMELS-SPAT dataset: {len(camelsspat_df)} watersheds")

# Display basic dataset information
print(f"\nDataset Overview:")
print(f"  Total watersheds: {len(camelsspat_df)}")
print(f"  Columns: {len(camelsspat_df.columns)}")
print(f"  Column names: {', '.join(camelsspat_df.columns[:8])}...")



camelsspat_df['latitude'] = camelsspat_df['Station_lat']
camelsspat_df['longitude'] = camelsspat_df['Station_lon']
camelsspat_df['drainage_area'] = camelsspat_df['Ref_shape_area_km2']

print(f"Coordinate extraction successful")
print(f"  Latitude range: {camelsspat_df['latitude'].min():.1f}° to {camelsspat_df['latitude'].max():.1f}°N")
print(f"  Longitude range: {camelsspat_df['longitude'].min():.1f}° to {camelsspat_df['longitude'].max():.1f}°W")
print(f"  Drainage area range: {camelsspat_df['drainage_area'].min():.0f} to {camelsspat_df['drainage_area'].max():.0f} km²")

# =============================================================================
# WATERSHED-SPECIFIC DATASET CHARACTERISTICS ANALYSIS
# =============================================================================

print(f"\nAnalyzing Watershed Dataset Characteristics...")

# Area-based watershed scale zones
area_zones = [
    (0, 100, 'Headwater'),
    (100, 1000, 'Meso-scale'),
    (1000, 5000, 'Macro-scale'),
    (5000, 50000, 'Large-scale')
]

camelsspat_df['area_class'] = 'Unknown'
for min_area, max_area, zone_name in area_zones:
    mask = (camelsspat_df['drainage_area'] >= min_area) & (camelsspat_df['drainage_area'] < max_area)
    camelsspat_df.loc[mask, 'area_class'] = zone_name

area_counts = camelsspat_df['area_class'].value_counts()
print(f"  Watershed scales: {len(area_counts)}")
print(f"    Most common: {area_counts.index[0]} ({area_counts.iloc[0]} watersheds)")

# Climate-based zones using aridity
if 'aridity' in camelsspat_df.columns:
    camelsspat_df['climate_class'] = 'Unknown'
    camelsspat_df.loc[camelsspat_df['aridity'] < 0.5, 'climate_class'] = 'Arid'
    camelsspat_df.loc[(camelsspat_df['aridity'] >= 0.5) & (camelsspat_df['aridity'] < 1.0), 'climate_class'] = 'Semi-arid'
    camelsspat_df.loc[(camelsspat_df['aridity'] >= 1.0) & (camelsspat_df['aridity'] < 2.0), 'climate_class'] = 'Sub-humid'
    camelsspat_df.loc[camelsspat_df['aridity'] >= 2.0, 'climate_class'] = 'Humid'
    
    climate_counts = camelsspat_df['climate_class'].value_counts()
    print(f"  Climate zones: {len(climate_counts)}")
    print(f"    Most common: {climate_counts.index[0]} ({climate_counts.iloc[0]} watersheds)")

# Regional analysis
if 'region' in camelsspat_df.columns:
    region_counts = camelsspat_df['region'].value_counts()
    print(f"  Regions: {len(region_counts)}")
    print(f"    Largest region: {region_counts.index[0]} ({region_counts.iloc[0]} watersheds)")

# Climate characteristics
if 'p_mean' in camelsspat_df.columns:
    precip_stats = camelsspat_df['p_mean'].describe()
    print(f"  Precipitation range: {precip_stats['min']:.0f} to {precip_stats['max']:.0f} mm/yr")

if 't_mean' in camelsspat_df.columns:
    temp_stats = camelsspat_df['t_mean'].describe()
    print(f"  Temperature range: {temp_stats['min']:.1f} to {temp_stats['max']:.1f} °C")

# Streamflow characteristics
if 'q_mean' in camelsspat_df.columns:
    flow_stats = camelsspat_df['q_mean'].describe()
    print(f"  Mean streamflow range: {flow_stats['min']:.1f} to {flow_stats['max']:.1f} m³/s")

# =============================================================================
# CAMELS-SPAT DATASET VISUALIZATION
# =============================================================================

print(f"\nCreating CAMELS-SPAT Dataset Overview Visualization...")

# Create comprehensive watershed dataset overview
fig, axes = plt.subplots(1, 3, figsize=(20, 12))

# 1. North American watershed distribution map
ax1 = axes[0]
scatter = ax1.scatter(camelsspat_df['longitude'], camelsspat_df['latitude'], 
                     c=camelsspat_df['drainage_area'], cmap='viridis', 
                     alpha=0.7, s=40, edgecolors='black', linewidth=0.5, norm=plt.Normalize(vmin=10, vmax=2000))
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title(f'CAMELS-SPAT Watershed Distribution\n({len(camelsspat_df)} watersheds)')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-130, -65)
ax1.set_ylim(25, 50)  # Focus on CONUS

# Add colorbar for drainage area
cbar = plt.colorbar(scatter, ax=ax1)
cbar.set_label('Drainage Area (km²)')

# 2. Watershed scale distribution
ax2 = axes[1]
if 'scale' in camelsspat_df.columns:
    scale_counts = camelsspat_df['scale'].value_counts()
else:
    scale_counts = camelsspat_df['area_class'].value_counts()

bars = ax2.bar(range(len(scale_counts)), scale_counts.values, 
               color='lightcoral', alpha=0.7, edgecolor='black')
ax2.set_xticks(range(len(scale_counts)))
ax2.set_xticklabels(scale_counts.index, rotation=45, ha='right')
ax2.set_ylabel('Number of Watersheds')
ax2.set_title('Watersheds by Scale')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, count in zip(bars, scale_counts.values):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
            str(count), ha='center', va='bottom', fontweight='bold')

# 3. Climate distribution
ax3 = axes[2]
if 'climate_class' in camelsspat_df.columns:
    climate_counts = camelsspat_df['climate_class'].value_counts()
    colors = ['brown', 'orange', 'lightgreen', 'blue']
    bars = ax3.bar(range(len(climate_counts)), climate_counts.values, 
                   color=colors[:len(climate_counts)], alpha=0.7, edgecolor='black')
    ax3.set_xticks(range(len(climate_counts)))
    ax3.set_xticklabels(climate_counts.index, rotation=45, ha='right')
    ax3.set_ylabel('Number of Watersheds')
    ax3.set_title('Watersheds by Climate')
    ax3.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, count in zip(bars, climate_counts.values):
        ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                str(count), ha='center', va='bottom', fontweight='bold')

plt.suptitle('CAMELS-SPAT Watershed Dataset - Overview', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Step 2: Automated CONFLUENCE Configuration and Batch Processing

Building on the dataset analysis and default configuration from Step 1, this step demonstrates automated large sample processing using the `run_watersheds_camelsspat.py` script. This script performs two key functions:

**Configuration Generation**: The script reads the CAMELS-Spat database and automatically creates individual CONFLUENCE configuration files for each site. Each configuration is customized with site-specific parameters including domain coordinates, bounding box definitions, and unique identifiers, while maintaining consistent model settings across all basins.

**Batch Job Submission**: The script submits SLURM jobs to execute the complete CONFLUENCE workflow for each basin in parallel. Each job processes geographic data, prepares meteorological forcing, processes FLUXNET observations, runs the hydrological model, and generates standardized output files.

This automated approach scales CONFLUENCE from single-domain modeling to systematic multi-site analysis across 1000+ watersheds

In [ ]:
def run_camelsspat_script_from_notebook():
    """
    Execute the run_watersheds_camelsspat-3.py script from within the notebook
    """
    print(f"\n🌊 Executing CAMELS-SPAT Large Sample Streamflow Processing Script...")
    
    script_path = "./run_watersheds_camelsspat-3.py"
    
    if not Path(script_path).exists():
        print(f"❌ Script not found: {script_path}")
        return False
    
    print(f"   📝 Script location: {script_path}")
    print(f"   🎯 Target watersheds: {len(selected_watersheds)} CAMELS-SPAT basins")
    print(f"   ⏰ Processing started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    try:
        # Create a process with interactive input automation
        process = subprocess.Popen(
            ['python', script_path],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True
        )
        
        # Prepare automated responses for the script prompts
        automated_inputs = [
            'n',  # Don't reload shapefile information
            'all',  # Process all scales
            str(streamflow_config['max_watersheds']),  # Number of watersheds to process
            'y' if not streamflow_config.get('dry_run_mode', False) else 'dry'  # Submit jobs or dry run
        ]
        
        input_string = '\n'.join(automated_inputs) + '\n'
        
        # Send automated responses
        stdout, stderr = process.communicate(input=input_string)
        
        # Print the output
        if stdout:
            print("📋 Script Output:")
            for line in stdout.split('\n'):
                if line.strip():
                    print(f"   {line}")
        
        if stderr:
            print("⚠️  Script Warnings/Errors:")
            for line in stderr.split('\n'):
                if line.strip():
                    print(f"   {line}")
        
        if process.returncode == 0:
            print(f"✅ CAMELS-SPAT processing script completed successfully")
            return True
        else:
            print(f"❌ Script failed with return code: {process.returncode}")
            return False
            
    except Exception as e:
        print(f"❌ Error running script: {e}")
        return False
        
# Execute the CAMELS-SPAT processing script
script_success = run_camelsspat_script_from_notebook()


## Step 3: Multi-Basin Streamflow Validation and Regional Analysis
Having executed large sample streamflow modeling, we now demonstrate the analytical power that emerges from systematic multi-basin streamflow validation using CAMELS-SPAT observations. This step showcases comprehensive watershed response evaluation, regional performance assessment, and integrated process validation—the scientific culmination of our entire CONFLUENCE tutorial series.
Streamflow Science Evolution: Case Studies → Systematic Watershed Understanding
Traditional Streamflow Validation: Individual basin model evaluation and calibration

Basin-specific parameter tuning with limited transferability to other watersheds
Difficulty separating universal hydrological principles from local environmental effects
Manual comparison across different studies and modeling approaches
Limited statistical power for robust hydrological process generalization

Large Sample Streamflow Validation: Systematic multi-basin hydrological analysis

Continental-scale pattern recognition across climate, topography, and scale gradients
Statistical hypothesis testing for hydrological process representations with robust sample sizes
Process universality assessment distinguishing general vs. basin-specific hydrological behaviors
Model transferability evaluation across diverse continental watershed environments

Comprehensive Multi-Basin Analysis Framework
Tier 1: Watershed Domain Spatial Overview

Automated discovery of completed streamflow modeling domains across environmental gradients
Processing status assessment including simulation completion, routing success, and observation availability
Continental spatial distribution showing streamflow modeling coverage across physiographic regions
Scale-based analysis revealing streamflow modeling performance across watershed size gradients

Tier 2: Integrated Streamflow Process Validation

Hydrograph comparison: Comprehensive streamflow time series validation across diverse watersheds
Multi-metric evaluation: Nash-Sutcliffe efficiency, Kling-Gupta efficiency, bias, and correlation assessment
Flow signature analysis: Characteristic watershed response patterns and hydrological behavior
Seasonal performance evaluation: Assessment across different hydrological seasons and flow conditions


In [ ]:
def discover_completed_streamflow_domains():
    """
    Discover all completed CAMELS-SPAT domain directories and their streamflow outputs
    """
    print(f"\n🔍 Discovering Completed CAMELS-SPAT Streamflow Modeling Domains...")
    
    # Base data directory pattern
    base_path = Path(streamflow_config['base_data_path'])
    domain_pattern = str(base_path / "domain_*")
    
    # Find all domain directories
    domain_dirs = glob.glob(domain_pattern)
    
    print(f"   📁 Found {len(domain_dirs)} total domain directories")
    
    completed_domains = []
    
    for domain_dir in domain_dirs:
        domain_path = Path(domain_dir)
        domain_name = domain_path.name.replace('domain_', '')
        
        # Check if this is a CAMELS-SPAT domain (should match our selected watersheds)
        if any(domain_name.startswith(ws) for ws in selected_watersheds['ID'].values):
            
            # Check for key output files
            shapefile_path = domain_path / "shapefiles" / "river_basins"
            simulation_dir = domain_path / "simulations"
            obs_dir = domain_path / "observations" / "streamflow" / "preprocessed"
            
            domain_info = {
                'domain_name': domain_name,
                'domain_path': domain_path,
                'has_shapefile': shapefile_path.exists(),
                'shapefile_path': shapefile_path if shapefile_path.exists() else None,
                'has_simulations': simulation_dir.exists(),
                'simulation_path': simulation_dir if simulation_dir.exists() else None,
                'has_observations': obs_dir.exists(),
                'observation_path': obs_dir if obs_dir.exists() else None,
                'simulation_files': [],
                'streamflow_obs_file': None
            }
            
            # Find simulation output files
            if simulation_dir.exists():
                # Look for SUMMA outputs
                summa_files = list(simulation_dir.glob("**/SUMMA/*.nc"))
                # Look for mizuRoute outputs (streamflow routing)
                mizuroute_files = list(simulation_dir.glob("**/mizuRoute/*.nc"))
                
                domain_info['simulation_files'] = summa_files + mizuroute_files
                domain_info['has_results'] = len(domain_info['simulation_files']) > 0
                domain_info['has_summa'] = len(summa_files) > 0
                domain_info['has_routing'] = len(mizuroute_files) > 0
            else:
                domain_info['has_results'] = False
                domain_info['has_summa'] = False
                domain_info['has_routing'] = False
            
            # Find observation files
            if obs_dir.exists():
                streamflow_files = list(obs_dir.glob("*streamflow*.csv"))
                if streamflow_files:
                    domain_info['streamflow_obs_file'] = streamflow_files[0]
            
            completed_domains.append(domain_info)
    
    print(f"   🌊 CAMELS-SPAT domains found: {len(completed_domains)}")
    print(f"   📊 Domains with shapefiles: {sum(1 for d in completed_domains if d['has_shapefile'])}")
    print(f"   📈 Domains with simulation results: {sum(1 for d in completed_domains if d['has_results'])}")
    print(f"   🌊 Domains with routing outputs: {sum(1 for d in completed_domains if d['has_routing'])}")
    print(f"   📋 Domains with observations: {sum(1 for d in completed_domains if d['has_observations'])}")
    
    return completed_domains

def create_streamflow_domain_overview_map(completed_domains):
    """
    Create an overview map showing all streamflow domain locations and their completion status
    """
    print(f"\n🗺️  Creating Streamflow Domain Overview Map...")
    
    # Create figure for overview map
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    
    # Map 1: Global overview with completion status
    ax1 = axes[0, 0]
    
    # Plot all selected sites
    if len(selected_watersheds) > 0:
        ax1.scatter(selected_watersheds['Lon'], selected_watersheds['Lat'], 
                   c='lightgray', alpha=0.5, s=30, label='Selected watersheds', marker='o')
    
    # Plot completed domains with different colors for different completion levels
    for domain in completed_domains:
        domain_name = domain['domain_name']
        
        # Find corresponding site in selected_watersheds
        site_row = None
        for _, row in selected_watersheds.iterrows():
            if domain_name.startswith(row['ID']):
                site_row = row
                break
        
        if site_row is not None:
            lat = site_row['Lat']
            lon = site_row['Lon']
            
            # Color based on completion status
            if domain['has_routing'] and domain['has_observations']:
                color = 'green'
                label = 'Complete with streamflow validation'
                marker = 's'
                size = 80
            elif domain['has_routing']:
                color = 'orange' 
                label = 'Routing complete'
                marker = '^'
                size = 60
            elif domain['has_results']:
                color = 'blue'
                label = 'Simulation complete'
                marker = 'D'
                size = 50
            else:
                color = 'red'
                label = 'Processing started'
                marker = 'v'
                size = 40
            
            ax1.scatter(lon, lat, c=color, s=size, marker=marker, alpha=0.8,
                       edgecolors='black', linewidth=0.5)
    
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.set_title('CAMELS-SPAT Streamflow Domain Processing Status Overview')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(-130, -60)  # Focus on North America
    ax1.set_ylim(25, 55)
    
    # Create custom legend
    legend_elements = [
        plt.scatter([], [], c='green', s=80, marker='s', label='Complete with validation'),
        plt.scatter([], [], c='orange', s=60, marker='^', label='Routing complete'),
        plt.scatter([], [], c='blue', s=50, marker='D', label='Simulation complete'),
        plt.scatter([], [], c='red', s=40, marker='v', label='Processing started'),
        plt.scatter([], [], c='lightgray', s=30, marker='o', label='Selected watersheds')
    ]
    ax1.legend(handles=legend_elements, loc='lower left')
    
    # Map 2: Completion statistics by scale
    ax2 = axes[0, 1]
    
    if len(selected_watersheds) > 0 and 'Scale' in selected_watersheds.columns:
        # Create scale-based completion analysis
        scale_completion = {}
        
        for domain in completed_domains:
            domain_name = domain['domain_name']
            
            # Find corresponding watershed
            site_row = None
            for _, row in selected_watersheds.iterrows():
                if domain_name.startswith(row['ID']):
                    site_row = row
                    break
            
            if site_row is not None:
                scale = site_row['Scale']
                
                if scale not in scale_completion:
                    scale_completion[scale] = {'total': 0, 'complete': 0, 'partial': 0}
                
                scale_completion[scale]['total'] += 1
                
                if domain['has_routing'] and domain['has_observations']:
                    scale_completion[scale]['complete'] += 1
                elif domain['has_results']:
                    scale_completion[scale]['partial'] += 1
        
        # Create stacked bar chart
        if scale_completion:
            scales = list(scale_completion.keys())
            complete_counts = [scale_completion[s]['complete'] for s in scales]
            partial_counts = [scale_completion[s]['partial'] for s in scales]
            pending_counts = [scale_completion[s]['total'] - 
                             scale_completion[s]['complete'] - 
                             scale_completion[s]['partial'] for s in scales]
            
            x_pos = range(len(scales))
            
            ax2.bar(x_pos, complete_counts, label='Complete', color='green', alpha=0.7)
            ax2.bar(x_pos, partial_counts, bottom=complete_counts, 
                   label='Partial', color='orange', alpha=0.7)
            ax2.bar(x_pos, pending_counts, 
                   bottom=[c+p for c,p in zip(complete_counts, partial_counts)], 
                   label='Pending', color='red', alpha=0.7)
            
            ax2.set_xticks(x_pos)
            ax2.set_xticklabels([s.capitalize() for s in scales])
            ax2.set_ylabel('Number of Watersheds')
            ax2.set_title('Processing Status by Watershed Scale')
            ax2.legend()
            ax2.grid(True, alpha=0.3, axis='y')
    
    # Map 3: Watershed area vs simulation status
    ax3 = axes[1, 0]
    
    domain_areas = []
    completion_status = []
    
    for domain in completed_domains:
        domain_name = domain['domain_name']
        
        # Find corresponding watershed
        site_row = None
        for _, row in selected_watersheds.iterrows():
            if domain_name.startswith(row['ID']):
                site_row = row
                break
        
        if site_row is not None and 'Area_km2' in site_row:
            area = site_row['Area_km2']
            domain_areas.append(area)
            
            # Determine completion status
            if domain['has_routing'] and domain['has_observations']:
                status = 'Complete'
                color = 'green'
            elif domain['has_routing']:
                status = 'Routing'
                color = 'orange'
            elif domain['has_results']:
                status = 'Simulation'
                color = 'blue'
            else:
                status = 'Started'
                color = 'red'
            
            completion_status.append(status)
            ax3.scatter(area, len(completion_status), c=color, alpha=0.7, s=60, 
                       edgecolors='black', linewidth=0.5)
    
    ax3.set_xlabel('Watershed Area (km²)')
    ax3.set_ylabel('Processing Order')
    ax3.set_title('Watershed Size vs Processing Status')
    ax3.set_xscale('log')
    ax3.grid(True, alpha=0.3)
    
    # Map 4: Processing summary statistics
    ax4 = axes[1, 1]
    
    # Summary statistics
    total_selected = len(selected_watersheds) if len(selected_watersheds) > 0 else 0
    total_discovered = len(completed_domains)
    total_with_results = sum(1 for d in completed_domains if d['has_results'])
    total_with_routing = sum(1 for d in completed_domains if d['has_routing'])
    total_with_obs = sum(1 for d in completed_domains if d['has_observations'])
    total_complete = sum(1 for d in completed_domains if d['has_routing'] and d['has_observations'])
    
    categories = ['Selected', 'Processing\nStarted', 'Simulation\nComplete', 'Routing\nComplete', 'Observations\nAvailable', 'Ready for\nValidation']
    counts = [total_selected, total_discovered, total_with_results, total_with_routing, total_with_obs, total_complete]
    colors = ['lightblue', 'yellow', 'blue', 'orange', 'cyan', 'green']
    
    bars = ax4.bar(categories, counts, color=colors, alpha=0.7, edgecolor='black')
    
    # Add value labels on bars
    for bar, count in zip(bars, counts):
        ax4.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.2,
                str(count), ha='center', va='bottom', fontweight='bold')
    
    ax4.set_ylabel('Number of Watersheds')
    ax4.set_title('Streamflow Modeling Processing Progress')
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.suptitle('CAMELS-SPAT Large Sample Streamflow Study - Domain Overview', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save the overview map
    overview_path = experiment_dir / 'plots' / 'streamflow_domain_overview_map.png'
    plt.savefig(overview_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Streamflow domain overview map saved: {overview_path}")
    
    return total_selected, total_discovered, total_with_results, total_with_routing, total_with_obs, total_complete

def extract_streamflow_results_from_domains(completed_domains):
    """
    Extract streamflow simulation results from all completed domains
    """
    print(f"\n🌊 Extracting Streamflow Results from Completed Domains...")
    
    streamflow_results = []
    processing_summary = {
        'total_domains': len(completed_domains),
        'domains_with_routing': 0,
        'domains_with_streamflow': 0,
        'failed_extractions': 0
    }
    
    for domain in completed_domains:
        if not domain['has_routing']:
            continue
            
        domain_name = domain['domain_name']
        processing_summary['domains_with_routing'] += 1
        
        try:
            print(f"   🔄 Processing {domain_name}...")
            
            # Find routing output files (mizuRoute)
            mizuroute_files = [f for f in domain['simulation_files'] if 'mizuRoute' in str(f)]
            
            if not mizuroute_files:
                print(f"     ❌ No mizuRoute files found")
                processing_summary['failed_extractions'] += 1
                continue
            
            # Use the first mizuRoute file
            output_file = mizuroute_files[0]
            
            # Load the netCDF file
            ds = xr.open_dataset(output_file)
            
            # Look for streamflow variables
            streamflow_vars = {}
            
            # Common mizuRoute streamflow variable names
            potential_vars = ['IRFroutedRunoff', 'routedRunoff', 'discharge', 'streamflow']
            
            for var in potential_vars:
                if var in ds.data_vars:
                    streamflow_vars['discharge'] = var
                    break
            
            if not streamflow_vars:
                print(f"     ⚠️  No streamflow variables found in {output_file.name}")
                available_vars = list(ds.data_vars.keys())
                print(f"     Available variables: {available_vars[:5]}...")
                processing_summary['failed_extractions'] += 1
                continue
            
            print(f"     🌊 Using streamflow variable: {streamflow_vars['discharge']}")
            
            # Extract streamflow data
            streamflow_var = streamflow_vars['discharge']
            streamflow_data = ds[streamflow_var]
            
            # Handle multi-dimensional data (time x reaches)
            if len(streamflow_data.dims) > 1:
                # Find the time dimension
                time_dim = 'time'
                reach_dims = [dim for dim in streamflow_data.dims if dim != time_dim]
                
                if reach_dims:
                    reach_dim = reach_dims[0]
                    # Use the last reach (often the outlet)
                    outlet_idx = streamflow_data.sizes[reach_dim] - 1
                    streamflow_data = streamflow_data.isel({reach_dim: outlet_idx})
                    print(f"     📍 Using outlet reach (index {outlet_idx})")
            
            # Convert to pandas Series
            streamflow_series = streamflow_data.to_pandas()
            
            # Handle unit conversion if needed (assume m³/s is correct)
            # Remove any negative values (set to 0)
            streamflow_series = streamflow_series.clip(lower=0)
            
            # Get site information
            site_row = None
            for _, row in selected_watersheds.iterrows():
                if domain_name.startswith(row['ID']):
                    site_row = row
                    break
            
            if site_row is None:
                print(f"     ⚠️  Site information not found for {domain_name}")
                continue
            
            # Calculate streamflow statistics
            streamflow_stats = {
                'mean_flow': streamflow_series.mean(),
                'max_flow': streamflow_series.max(),
                'min_flow': streamflow_series.min(),
                'std_flow': streamflow_series.std(),
                'flow_variability': streamflow_series.std() / streamflow_series.mean() if streamflow_series.mean() > 0 else np.nan
            }
            
            # Calculate flow percentiles
            percentiles = [5, 25, 50, 75, 95]
            for p in percentiles:
                streamflow_stats[f'q{p}'] = streamflow_series.quantile(p/100)
            
            # Store results
            result = {
                'domain_name': domain_name,
                'watershed_id': site_row['ID'],
                'latitude': site_row['Lat'],
                'longitude': site_row['Lon'],
                'area_km2': site_row.get('Area_km2', np.nan),
                'scale': site_row.get('Scale', 'unknown'),
                'streamflow_timeseries': streamflow_series,
                'data_period': f"{streamflow_series.index.min()} to {streamflow_series.index.max()}",
                'data_points': len(streamflow_series),
                'streamflow_variable': streamflow_var,
                'output_file': str(output_file)
            }
            
            # Add statistics
            result.update(streamflow_stats)
            
            streamflow_results.append(result)
            processing_summary['domains_with_streamflow'] += 1
            
            print(f"     ✅ Streamflow extracted: {result['mean_flow']:.2f} m³/s (range: {result['min_flow']:.2f}-{result['max_flow']:.2f})")
            
        except Exception as e:
            print(f"     ❌ Error processing {domain_name}: {e}")
            processing_summary['failed_extractions'] += 1
    
    print(f"\n🌊 Streamflow Extraction Summary:")
    print(f"   Total domains: {processing_summary['total_domains']}")
    print(f"   Domains with routing: {processing_summary['domains_with_routing']}")
    print(f"   Successful extractions: {processing_summary['domains_with_streamflow']}")
    print(f"   Failed extractions: {processing_summary['failed_extractions']}")
    
    return streamflow_results, processing_summary

def load_camelsspat_observations(completed_domains):
    """
    Load CAMELS-SPAT observation data for streamflow validation
    """
    print(f"\n📥 Loading CAMELS-SPAT Streamflow Observation Data...")
    
    camelsspat_obs = {}
    obs_summary = {
        'sites_found': 0,
        'sites_with_streamflow': 0,
        'total_observations': 0
    }
    
    # Look for processed CAMELS-SPAT observation data in domain directories
    for domain in completed_domains:
        if not domain['has_observations']:
            continue
            
        domain_name = domain['domain_name']
        
        try:
            print(f"   📊 Loading {domain_name}...")
            
            obs_summary['sites_found'] += 1
            
            # Load streamflow observations
            if domain['streamflow_obs_file']:
                obs_df = pd.read_csv(domain['streamflow_obs_file'])
                
                # Find time and discharge columns
                time_col = None
                for col in ['datetime', 'date', 'time']:
                    if col in obs_df.columns:
                        time_col = col
                        break
                
                discharge_col = None
                for col in ['discharge_cms', 'streamflow', 'flow', 'Q']:
                    if col in obs_df.columns:
                        discharge_col = col
                        break
                
                if time_col and discharge_col:
                    obs_df[time_col] = pd.to_datetime(obs_df[time_col])
                    obs_df.set_index(time_col, inplace=True)
                    
                    streamflow_obs = obs_df[discharge_col].dropna()
                    
                    if len(streamflow_obs) > 0:
                        # Calculate streamflow statistics
                        obs_stats = {
                            'mean_flow': streamflow_obs.mean(),
                            'max_flow': streamflow_obs.max(),
                            'min_flow': streamflow_obs.min(),
                            'std_flow': streamflow_obs.std(),
                            'flow_variability': streamflow_obs.std() / streamflow_obs.mean() if streamflow_obs.mean() > 0 else np.nan
                        }
                        
                        # Calculate flow percentiles
                        percentiles = [5, 25, 50, 75, 95]
                        for p in percentiles:
                            obs_stats[f'q{p}'] = streamflow_obs.quantile(p/100)
                        
                        # Store observation data
                        site_obs = {
                            'streamflow_timeseries': streamflow_obs,
                            'data_period': f"{streamflow_obs.index.min()} to {streamflow_obs.index.max()}",
                            'data_points': len(streamflow_obs)
                        }
                        
                        # Add statistics
                        site_obs.update(obs_stats)
                        
                        # Add site metadata
                        site_row = None
                        for _, row in selected_watersheds.iterrows():
                            if domain_name.startswith(row['ID']):
                                site_row = row
                                break
                        
                        if site_row is not None:
                            site_obs['latitude'] = site_row['Lat']
                            site_obs['longitude'] = site_row['Lon']
                            site_obs['area_km2'] = site_row.get('Area_km2', np.nan)
                            site_obs['scale'] = site_row.get('Scale', 'unknown')
                            site_obs['watershed_id'] = site_row['ID']
                        
                        camelsspat_obs[domain_name] = site_obs
                        
                        obs_summary['sites_with_streamflow'] += 1
                        obs_summary['total_observations'] += len(streamflow_obs)
                        
                        print(f"     🌊 Streamflow obs: {streamflow_obs.mean():.2f} m³/s (range: {streamflow_obs.min():.2f}-{streamflow_obs.max():.2f}) ({len(streamflow_obs)} points)")
                else:
                    print(f"     ⚠️ Could not find time/discharge columns in observation file")
            
        except Exception as e:
            print(f"     ❌ Error loading {domain_name}: {e}")
    
    print(f"\n🌊 CAMELS-SPAT Observation Summary:")
    print(f"   Sites with observation files: {obs_summary['sites_found']}")
    print(f"   Sites with streamflow observations: {obs_summary['sites_with_streamflow']}")
    print(f"   Total streamflow observations: {obs_summary['total_observations']}")
    
    return camelsspat_obs, obs_summary

def create_streamflow_comparison_analysis(streamflow_results, camelsspat_obs):
    """
    Create comprehensive streamflow comparison analysis between simulated and observed
    """
    print(f"\n🌊 Creating Streamflow Comparison Analysis...")
    
    # Find sites with both simulated and observed data
    common_sites = []
    
    for sim_result in streamflow_results:
        domain_name = sim_result['domain_name']
        
        if domain_name in camelsspat_obs:
            # Align time periods
            sim_flow = sim_result['streamflow_timeseries']
            obs_flow = camelsspat_obs[domain_name]['streamflow_timeseries']
            
            # Find common time period
            common_start = max(sim_flow.index.min(), obs_flow.index.min())
            common_end = min(sim_flow.index.max(), obs_flow.index.max())
            
            if common_start < common_end:
                # Resample to daily and align
                sim_daily = sim_flow.resample('D').mean().loc[common_start:common_end]
                obs_daily = obs_flow.resample('D').mean().loc[common_start:common_end]
                
                # Remove NaN values
                valid_mask = ~(sim_daily.isna() | obs_daily.isna())
                sim_valid = sim_daily[valid_mask]
                obs_valid = obs_daily[valid_mask]
                
                if len(sim_valid) > 50:  # Need minimum data for meaningful comparison
                    
                    # Calculate performance metrics
                    def calculate_nse(obs, sim):
                        return 1 - ((obs - sim) ** 2).sum() / ((obs - obs.mean()) ** 2).sum()
                    
                    def calculate_kge(obs, sim):
                        # Kling-Gupta Efficiency
                        r = np.corrcoef(obs, sim)[0, 1]
                        alpha = sim.std() / obs.std()
                        beta = sim.mean() / obs.mean()
                        kge = 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
                        return kge
                    
                    # Performance metrics
                    nse = calculate_nse(obs_valid, sim_valid)
                    rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
                    bias = (sim_valid - obs_valid).mean()
                    pbias = 100 * bias / obs_valid.mean() if obs_valid.mean() > 0 else np.nan
                    
                    # Correlation
                    try:
                        correlation = obs_valid.corr(sim_valid)
                        if pd.isna(correlation):
                            correlation = 0.0
                    except:
                        correlation = 0.0
                    
                    # KGE
                    try:
                        kge = calculate_kge(obs_valid.values, sim_valid.values)
                        if pd.isna(kge):
                            kge = -999
                    except:
                        kge = -999
                    
                    common_site = {
                        'domain_name': domain_name,
                        'watershed_id': sim_result['watershed_id'],
                        'latitude': sim_result['latitude'],
                        'longitude': sim_result['longitude'],
                        'area_km2': sim_result['area_km2'],
                        'scale': sim_result['scale'],
                        'sim_flow': sim_valid,
                        'obs_flow': obs_valid,
                        'sim_mean': sim_valid.mean(),
                        'obs_mean': obs_valid.mean(),
                        'nse': nse,
                        'kge': kge,
                        'rmse': rmse,
                        'bias': bias,
                        'pbias': pbias,
                        'correlation': correlation,
                        'n_points': len(sim_valid),
                        'common_period': f"{common_start.date()} to {common_end.date()}"
                    }
                    
                    common_sites.append(common_site)
                    
                    print(f"   ✅ {domain_name}: NSE={nse:.3f}, KGE={kge:.3f}, r={correlation:.3f}, Bias={bias:+.2f} ({len(sim_valid)} points)")
    
    print(f"\n🌊 Streamflow Comparison Summary:")
    print(f"   Sites with both sim and obs: {len(common_sites)}")
    
    if len(common_sites) == 0:
        print("   ⚠️  No sites with overlapping sim/obs data for comparison")
        return None
    
    # Create comprehensive streamflow comparison visualization
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    
    # Scatter plot: Observed vs Simulated (top left)
    ax1 = axes[0, 0]
    
    all_obs = np.concatenate([site['obs_flow'].values for site in common_sites])
    all_sim = np.concatenate([site['sim_flow'].values for site in common_sites])
    
    ax1.scatter(all_obs, all_sim, alpha=0.3, s=8, c='blue')
    
    # 1:1 line
    min_val = min(all_obs.min(), all_sim.min())
    max_val = max(all_obs.max(), all_sim.max())
    ax1.plot([min_val, max_val], [min_val, max_val], 'k--', label='1:1 line')
    
    ax1.set_xlabel('Observed Streamflow (m³/s)')
    ax1.set_ylabel('Simulated Streamflow (m³/s)')
    ax1.set_title('All Sites: Simulated vs Observed Streamflow')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    
    # Add overall statistics
    overall_corr = np.corrcoef(all_obs, all_sim)[0,1] if len(all_obs) > 1 else 0
    overall_nse = 1 - ((all_obs - all_sim) ** 2).sum() / ((all_obs - all_obs.mean()) ** 2).sum()
    overall_bias = np.mean(all_sim - all_obs)
    
    stats_text = f'r = {overall_corr:.3f}\nNSE = {overall_nse:.3f}\nBias = {overall_bias:+.2f}'
    ax1.text(0.05, 0.95, stats_text, transform=ax1.transAxes,
             bbox=dict(facecolor='white', alpha=0.8), fontsize=10, verticalalignment='top')
    
    # Performance by watershed scale (top middle)
    ax2 = axes[0, 1]
    
    if any('scale' in site for site in common_sites):
        scale_stats = {}
        for site in common_sites:
            scale = site.get('scale', 'unknown')
            if scale not in scale_stats:
                scale_stats[scale] = {'nse': [], 'kge': [], 'corr': []}
            
            scale_stats[scale]['nse'].append(site['nse'])
            scale_stats[scale]['kge'].append(site['kge'])
            scale_stats[scale]['corr'].append(site['correlation'])
        
        # Plot NSE by scale
        scales = list(scale_stats.keys())
        nse_means = [np.mean(scale_stats[s]['nse']) for s in scales]
        nse_stds = [np.std(scale_stats[s]['nse']) for s in scales]
        
        x_pos = range(len(scales))
        bars = ax2.bar(x_pos, nse_means, yerr=nse_stds, capsize=5, alpha=0.7, color='skyblue')
        ax2.set_xticks(x_pos)
        ax2.set_xticklabels([s.capitalize() for s in scales])
        ax2.set_ylabel('Nash-Sutcliffe Efficiency')
        ax2.set_title('Streamflow Performance by Watershed Scale')
        ax2.grid(True, alpha=0.3, axis='y')
        ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5)
        
        # Add value labels
        for bar, mean_val in zip(bars, nse_means):
            ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.02,
                    f'{mean_val:.2f}', ha='center', va='bottom', fontsize=9)
    
    # Performance vs watershed area (top right)
    ax3 = axes[0, 2]
    
    areas = [site['area_km2'] for site in common_sites if not np.isnan(site['area_km2'])]
    nses = [site['nse'] for site in common_sites if not np.isnan(site['area_km2'])]
    
    if areas and nses:
        scatter3 = ax3.scatter(areas, nses, alpha=0.7, s=40, c='green')
        ax3.set_xlabel('Watershed Area (km²)')
        ax3.set_ylabel('Nash-Sutcliffe Efficiency')
        ax3.set_title('Performance vs Watershed Size')
        ax3.grid(True, alpha=0.3)
        ax3.set_xscale('log')
        ax3.axhline(y=0, color='red', linestyle='--', alpha=0.5)
        ax3.axhline(y=0.5, color='orange', linestyle='--', alpha=0.5, label='NSE = 0.5')
        ax3.legend()
    
    # Bias distribution (bottom left)
    ax4 = axes[1, 0]
    
    biases = [site['bias'] for site in common_sites]
    ax4.hist(biases, bins=15, color='orange', alpha=0.7, edgecolor='black')
    ax4.axvline(x=0, color='red', linestyle='--', label='Zero bias')
    ax4.set_xlabel('Bias (m³/s)')
    ax4.set_ylabel('Number of Watersheds')
    ax4.set_title('Distribution of Streamflow Bias')
    ax4.legend()
    ax4.grid(True, alpha=0.3, axis='y')
    
    # KGE vs NSE comparison (bottom middle)
    ax5 = axes[1, 1]
    
    nse_vals = [site['nse'] for site in common_sites]
    kge_vals = [site['kge'] for site in common_sites if site['kge'] != -999]
    
    if len(kge_vals) > 0:
        ax5.scatter(nse_vals[:len(kge_vals)], kge_vals, alpha=0.7, s=40, c='purple')
        ax5.set_xlabel('Nash-Sutcliffe Efficiency')
        ax5.set_ylabel('Kling-Gupta Efficiency')
        ax5.set_title('NSE vs KGE Performance')
        ax5.grid(True, alpha=0.3)
        
        # Add reference lines
        ax5.axhline(y=0, color='red', linestyle='--', alpha=0.5)
        ax5.axvline(x=0, color='red', linestyle='--', alpha=0.5)
        ax5.plot([-1, 1], [-1, 1], 'k--', alpha=0.3, label='1:1 line')
        ax5.legend()
    
    # Performance summary (bottom right)
    ax6 = axes[1, 2]
    
    # Create performance categories
    perf_categories = {
        'Excellent (NSE > 0.75)': len([s for s in common_sites if s['nse'] > 0.75]),
        'Good (0.5 < NSE ≤ 0.75)': len([s for s in common_sites if 0.5 < s['nse'] <= 0.75]),
        'Satisfactory (0.2 < NSE ≤ 0.5)': len([s for s in common_sites if 0.2 < s['nse'] <= 0.5]),
        'Unsatisfactory (NSE ≤ 0.2)': len([s for s in common_sites if s['nse'] <= 0.2])
    }
    
    categories = list(perf_categories.keys())
    counts = list(perf_categories.values())
    colors = ['darkgreen', 'green', 'yellow', 'red']
    
    bars = ax6.bar(range(len(categories)), counts, color=colors, alpha=0.7, edgecolor='black')
    ax6.set_xticks(range(len(categories)))
    ax6.set_xticklabels([c.split('(')[0].strip() for c in categories], rotation=45, ha='right')
    ax6.set_ylabel('Number of Watersheds')
    ax6.set_title('Performance Category Distribution')
    ax6.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar, count in zip(bars, counts):
        ax6.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
                str(count), ha='center', va='bottom', fontweight='bold')
    
    plt.suptitle('CAMELS-SPAT Large Sample Streamflow Comparison Analysis', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save comparison plot
    comparison_path = experiment_dir / 'plots' / 'streamflow_comparison_analysis.png'
    plt.savefig(comparison_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Streamflow comparison analysis saved: {comparison_path}")
    
    # Create spatial performance map
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    # Map 1: NSE spatial distribution
    ax1 = axes[0]
    
    lats = [site['latitude'] for site in common_sites]
    lons = [site['longitude'] for site in common_sites]
    nse_values = [site['nse'] for site in common_sites]
    
    scatter1 = ax1.scatter(lons, lats, c=nse_values, cmap='RdYlGn', s=100, 
                          vmin=-0.5, vmax=1.0, edgecolors='black', linewidth=0.5)
    
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.set_title('Streamflow Model Performance: NSE')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(-130, -60)
    ax1.set_ylim(25, 55)
    
    # Add colorbar
    cbar1 = plt.colorbar(scatter1, ax=ax1)
    cbar1.set_label('Nash-Sutcliffe Efficiency')
    
    # Map 2: Bias spatial distribution
    ax2 = axes[1]
    
    bias_values = [site['bias'] for site in common_sites]
    max_abs_bias = max(abs(min(bias_values)), abs(max(bias_values)))
    
    scatter2 = ax2.scatter(lons, lats, c=bias_values, cmap='RdBu_r', s=100,
                          vmin=-max_abs_bias, vmax=max_abs_bias, 
                          edgecolors='black', linewidth=0.5)
    
    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Latitude')
    ax2.set_title('Streamflow Model Performance: Bias (Sim - Obs)')
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(-130, -60)
    ax2.set_ylim(25, 55)
    
    # Add colorbar
    cbar2 = plt.colorbar(scatter2, ax=ax2)
    cbar2.set_label('Bias (m³/s)')
    
    plt.suptitle('CAMELS-SPAT Large Sample Streamflow Performance - Spatial Distribution', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save spatial analysis
    spatial_path = experiment_dir / 'plots' / 'streamflow_spatial_performance.png'
    plt.savefig(spatial_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Streamflow spatial performance map saved: {spatial_path}")
    
    return common_sites

# Execute Step 3 Analysis
print(f"\n🔍 Step 3.1: Streamflow Domain Discovery and Overview")

# Discover completed domains
completed_domains = discover_completed_streamflow_domains()

# Create domain overview map
if len(completed_domains) > 0:
    total_selected, total_discovered, total_with_results, total_with_routing, total_with_obs, total_complete = create_streamflow_domain_overview_map(completed_domains)
else:
    print("   ⚠️ No completed domains found for overview map")
    total_selected = len(selected_watersheds) if 'selected_watersheds' in locals() else 0
    total_discovered = total_with_results = total_with_routing = total_with_obs = total_complete = 0

print(f"\n🌊 Step 3.2: Streamflow Results Extraction")

# Extract streamflow results from simulations
if len(completed_domains) > 0:
    streamflow_results, streamflow_processing_summary = extract_streamflow_results_from_domains(completed_domains)
    
    # Load CAMELS-SPAT observations
    camelsspat_obs, obs_summary = load_camelsspat_observations(completed_domains)
else:
    print("   ⚠️ No completed domains available for analysis")
    streamflow_results = []
    camelsspat_obs = {}
    streamflow_processing_summary = {'domains_with_streamflow': 0}
    obs_summary = {'sites_with_streamflow': 0}

print(f"\n🌊 Step 3.3: Streamflow Comparison Analysis")

# Create streamflow comparison analysis
if streamflow_results and camelsspat_obs:
    common_sites = create_streamflow_comparison_analysis(streamflow_results, camelsspat_obs)
else:
    print("   ⚠️  Insufficient data for streamflow comparison analysis")
    common_sites = None

# Create final summary report
print(f"\n📋 Creating Final CAMELS-SPAT Streamflow Study Summary Report...")

summary_report_path = experiment_dir / 'reports' / 'camelsspat_final_report.txt'

with open(summary_report_path, 'w') as f:
    f.write("CAMELS-SPAT Large Sample Streamflow Study - Final Analysis Report\n")
    f.write("=" * 68 + "\n\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    f.write("PROCESSING SUMMARY:\n")
    f.write(f"  Watersheds selected for analysis: {total_selected}\n")
    f.write(f"  Processing initiated: {total_discovered}\n")
    f.write(f"  Simulation results available: {total_with_results}\n")
    f.write(f"  Routing outputs available: {total_with_routing}\n")
    f.write(f"  Observations available: {total_with_obs}\n")
    f.write(f"  Complete streamflow validation: {total_complete}\n")
    f.write(f"  Streamflow extractions successful: {streamflow_processing_summary['domains_with_streamflow']}\n")
    f.write(f"  CAMELS-SPAT observations available: {obs_summary['sites_with_streamflow']}\n")
    
    if common_sites:
        f.write(f"  Sites with sim/obs comparison: {len(common_sites)}\n\n")
        
        # Streamflow performance summary
        nse_values = [site['nse'] for site in common_sites]
        kge_values = [site['kge'] for site in common_sites if site['kge'] != -999]
        bias_values = [site['bias'] for site in common_sites]
        corr_values = [site['correlation'] for site in common_sites]
        
        f.write("STREAMFLOW PERFORMANCE SUMMARY:\n")
        f.write(f"  Mean NSE: {np.mean(nse_values):.3f} ± {np.std(nse_values):.3f}\n")
        if kge_values:
            f.write(f"  Mean KGE: {np.mean(kge_values):.3f} ± {np.std(kge_values):.3f}\n")
        f.write(f"  Mean correlation: {np.mean(corr_values):.3f} ± {np.std(corr_values):.3f}\n")
        f.write(f"  Mean bias: {np.mean(bias_values):+.2f} ± {np.std(bias_values):.2f} m³/s\n\n")
        
        # Performance categories
        excellent = len([s for s in common_sites if s['nse'] > 0.75])
        good = len([s for s in common_sites if 0.5 < s['nse'] <= 0.75])
        satisfactory = len([s for s in common_sites if 0.2 < s['nse'] <= 0.5])
        unsatisfactory = len([s for s in common_sites if s['nse'] <= 0.2])
        
        f.write("PERFORMANCE CATEGORIES:\n")
        f.write(f"  Excellent (NSE > 0.75): {excellent} watersheds\n")
        f.write(f"  Good (0.5 < NSE ≤ 0.75): {good} watersheds\n")
        f.write(f"  Satisfactory (0.2 < NSE ≤ 0.5): {satisfactory} watersheds\n")
        f.write(f"  Unsatisfactory (NSE ≤ 0.2): {unsatisfactory} watersheds\n\n")
        
        f.write("BEST PERFORMING WATERSHEDS (by NSE):\n")
        sorted_sites = sorted(common_sites, key=lambda x: x['nse'], reverse=True)
        for i, site in enumerate(sorted_sites[:5]):
            f.write(f"  {i+1}. {site['watershed_id']}: NSE={site['nse']:.3f}, KGE={site['kge']:.3f}, Area={site['area_km2']:.0f} km²\n")

print(f"✅ Final summary report saved: {summary_report_path}")

print(f"\n🎉 Step 3 Complete: CAMELS-SPAT Streamflow Validation Analysis")
print(f"   📁 Results saved to: {experiment_dir}")
print(f"   🌊 Streamflow domain overview: {total_complete}/{total_selected} watersheds with complete validation")

if common_sites:
    nse_values = [site['nse'] for site in common_sites]
    kge_values = [site['kge'] for site in common_sites if site['kge'] != -999]
    
    print(f"   📊 Streamflow analysis: {len(common_sites)} watersheds with sim/obs comparison")
    print(f"   📈 NSE performance: Mean = {np.mean(nse_values):.3f}")
    if kge_values:
        print(f"   📈 KGE performance: Mean = {np.mean(kge_values):.3f}")
else:
    print(f"   📈 Performance: Awaiting more simulation results")

print(f"\n✅ Large Sample CAMELS-SPAT Streamflow Analysis Complete!")
print(f"   🌊 Multi-basin streamflow hydrology validation achieved")
print(f"   📊 Statistical patterns identified across continental watershed gradients")
print(f"   🏔️ Tutorial series culmination: Point → Watershed → Continental → Multi-site analysis!")

# Tutorial 04c Summary: CAMELS Large Sample Streamflow Study

## Overview

This tutorial demonstrates large sample streamflow modeling across multiple watersheds using the CAMELS-SPAT dataset. It represents the culmination of the CONFLUENCE tutorial series, advancing from point-scale process validation to integrated watershed-scale hydrological analysis across diverse environmental gradients.

## Learning Objectives

Students will configure CONFLUENCE for multi-basin streamflow analysis using standardized watershed data, execute systematic streamflow modeling across environmental gradients, validate simulated streamflow against observed discharge using multiple performance metrics, analyze regional patterns in model performance, and demonstrate workflow automation for large sample hydrological studies.

## Tutorial Structure

The tutorial begins with analysis of the CAMELS-SPAT watershed database covering diverse North American basins and creation of standardized CONFLUENCE configuration templates. The second component involves automated multi-basin processing execution across selected watersheds, demonstrating batch processing capabilities and monitoring systems. The final section focuses on streamflow validation through extraction and comparison of simulated versus observed time series, calculation of performance metrics including Nash-Sutcliffe efficiency and Kling-Gupta efficiency, spatial performance mapping, and comprehensive analysis reporting.

## Scientific Significance

This tutorial addresses fundamental questions in watershed hydrology including controls on streamflow generation, model transferability across environmental gradients, and systematic evaluation of hydrological process representations. The large sample approach enables robust statistical analysis and identification of universal versus site-specific hydrological behaviors, contributing to improved understanding of continental-scale watershed function and enhanced predictive capabilities for water resources management.